In [83]:
import osmnx as ox, networkx as nx, geopandas as gpd, matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon, LineString
from descartes import PolygonPatch
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
ox.config(use_cache=True, log_console=True)
ox.__version__

C:\Users\smdhk\anaconda3\envs\ox\lib\site-packages\osmnx\utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


'1.2.2'

In [84]:
import pandas as pd
import numpy as np

df = pd.read_csv('송파구.csv', encoding='cp949', index_col=0)
idx = df[df['대여 대여소번호']==df['반납대여소번호']].index
df.drop(idx, inplace=True)
df.drop(['0'],axis=1, inplace=True)

In [ ]:
#df2 = df[0:10000]

In [2]:
import osmnx as ox

place = '서울, 대한민국'
G = ox.graph_from_place(place, network_type='drive', simplify=False)

In [90]:
Gs = ox.utils_graph.get_largest_component(G, strongly=True)

In [93]:
df.iloc[77]

대여 대여소번호    4891.000000
반납대여소번호     4453.000000
대여소 위도        37.484554
대여소 경도       127.149193
반납소 위도        37.474346
반납소 경도       127.143265
Name: 77, dtype: float64

In [99]:
from tqdm import tqdm



df2 = df
df2.reset_index(drop=True, inplace=True)
road_li = [] #도로 기준 최단 거리

for i in tqdm(range(len(df2))):
    if (df2['대여소 경도'][i] == df2['반납소 경도'][i]) & (df2['대여소 위도'][i] == df2['반납소 위도'][i]):
        pass
    else:
        orig_node = ox.nearest_nodes(Gs, X=df2['대여소 경도'][i], Y=df2['대여소 위도'][i]) #출발지
        dest_node = ox.nearest_nodes(Gs, X=df2['반납소 경도'][i], Y=df2['반납소 위도'][i]) #목적지
        len_road = nx.shortest_path_length(Gs, orig_node, dest_node, weight='length')
        road_li.append(str(round(len_road,1))+'m')


  0%|          | 89/23671 [02:53<12:44:37,  1.95s/it]


KeyboardInterrupt: 

In [95]:
dest_node

4599501933

In [ ]:
t = [] # 최단거리 bike 기준 시간
bike_sp = 15000 # 자전거 속도 15km/h

road_li.columns = ['최단거리']

for i in range(len(road_li)):
    etc = road_li['최단거리'][i] / bike_sp
    hour = int(etc)
    mint = int((etc - hour)*60)
    sec = int(((etc-hour)*60-mint)*60)
    t.append(hour, mint, sec)

road_li['최단시간'] = t

In [ ]:
df3 = pd.concat(df2, road_li, axis=1, ignore_index=True)
df3.to_csv('송파구.csv', encoding='utf8')